# Homework 2 (WIP)

* Implement an explicit Runge-Kutta integrator that takes an initial time step $h_0$ and an error tolerance $\epsilon$.
* You can use the Bogacki-Shampine method or any other method with an embedded error estimate.
* A step should be rejected if the local truncation error exceeds the tolerance.
* Test your method on the nonlinear equation
$$ \begin{bmatrix} \dot u_0 \\ \dot u_1 \end{bmatrix} = \begin{bmatrix} u_1 \\ k (1-u_0^2) u_1 - u_0 \end{bmatrix} $$
for $k=2$, $k=5$, and $k=20$.
* Make a work-precision diagram for your adaptive method and for constant step sizes.

In [ ]:
%precision 3
%matplotlib notebook

import numpy
from matplotlib import pyplot
pyplot.style.use('ggplot')

## Runge-Kutta Implementation

A work-in-progress adaptation of code from the lectures.

Incidentally, according to the internet, the test equation is the Van der Pol Equation:

$$x'' - k(1 - x^2)x' + x = 0$$

In [ ]:
def expm(A):
    """
    Compute the matrix exponential
    """
    L, X = numpy.linalg.eig(A)
    return X.dot(numpy.diag(numpy.exp(L))).dot(numpy.linalg.inv(X))

class linear:
    def __init__(self, A):
        self.A = A.copy()
    def f(self, t, u):
        return self.A.dot(u)
    def u(self, t, u0):
        t = numpy.array(t, ndmin=1)
        return [numpy.real(expm(self.A*s).dot(u0)) for s in t]

class hw_eqn:
    def __init__(self, k):
        self.k = k.copy()
    def f(self, t, u):
        return [u[1], k * (1 - u[0] ** 2) * u[1] - u[0]]
# TODO: add a solution
#     def u(self, t, u0):
#         t = numpy.array(t, ndmin=1)
#         return [numpy.real(expm(self.A*s).dot(u0)) for s in t]

def rk_butcher_4():
    '''
    Provides Butcher Table values for RK4.
    '''
    A = numpy.array([[0,0,0,0],[.5,0,0,0],[0,.5,0,0],[0,0,1,0]])
    b = numpy.array([1/6, 1/3, 1/3, 1/6])
    return A, b

def rk_butcher_bs3():
    '''
    Provides Butcher Table values for Bogacki–Shampine method.
    '''
    A = numpy.array([
        [0, 0, 0, 0], 
        [1/2, 0, 0, 0], 
        [0, 3/4, 0, 0], 
        [2/9, 1/3, 4/9, 0]])
    
    # b has two rows, (1) the one we'll propogate and (2) the error estimator
    # There are four stages, with 3rd-order accuracy from the propogated result.
    b = numpy.array([
        [2/9, 1/3, 4/9, 0], 
        [7/24, 1/4, 1/3, 1/8]])
    
    return A, b

def ode_rkexplicit(f, u0, tfinal=1, h=.1, eps=0.01):
    '''
    f: RHS function of the equation u' = f(t, u)
    u0: initial guess for u, an nx1 array
    tfinal: final time value
    h: initial time step size (default: 0.1)
    eps: error tolerance
    '''
    A, b = rk_butcher_bs3()

    c = numpy.sum(A, axis=1)
    s = len(c)
    u = u0.copy()
    t = 0
    hist = [(t,u0)]
    while t < tfinal:
        # Prepare the next time value
        if tfinal - t < 1.01*h:
            h = tfinal - t
            tnext = tfinal
        else:
            tnext = t + h
        
        # Ensure that we don't step beyond the allowed time interval
        h = min(h, tfinal - t)
        
        # First, calculate the sums of f(t + c_jh, Y_j) in fY
        fY = numpy.zeros((len(u0), s))
        for i in range(s):
            Yi = u.copy()
            for j in range(i):
                Yi += h * A[i,j] * fY[:,j]

            # Final Y_i value is available; apply f
            fY[:,i] = f(t + h*c[i], Yi)

        # Take the time step and advance u
        u += h * fY.dot(b)
        t = tnext
        
        # Record the time and its corresponding u approximation
        hist.append((t, u.copy()))

    return hist

# test = linear(numpy.array([[0, 1],[-1, 0]]))
# u0 = numpy.array([.5, 0])
# hist = ode_rkexplicit(test.f, u0, rk_butcher_bs3(), tfinal=50, h=.8)
# times = [t for t,u in hist]
# pyplot.plot(times, [u for t,u in hist], '.')
# pyplot.plot(times, test.u(times, u0));